In [7]:
# Imports
import os
from tqdm import tqdm
from flourish_child.models import Appointment as ChildAppointment
from edc_appointment.models import Appointment as CaregiverAppointment
from dateutil.relativedelta import relativedelta

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [8]:
# Query participants who have been sequentially enrolled
sq_enrolled = Cohort.objects.values(
    'subject_identifier').annotate(cohort_count=Count('subject_identifier')).filter(cohort_count__gt=1)
child_identifiers = sq_enrolled.values_list('subject_identifier', flat=True)
child_identifiers = set(child_identifiers)

In [9]:
for child in tqdm(child_identifiers):
    child_consent = CaregiverChildConsent.objects.filter(subject_identifier=child).latest('consent_datetime')
    fu_schedules = SubjectScheduleHistory.objects.filter(
        subject_identifier=child, schedule_name__icontains='fu').exclude(
        Q(schedule_name__icontains='qt') | Q(schedule_name__icontains='quart'))

    if fu_schedules.count() > 1:
        try:
            appt_1 = ChildAppointment.objects.get(
                subject_identifier=child, schedule_name=fu_schedules[0].schedule_name, visit_code_sequence=0)
        except ChildAppointment.DoesNotExist:
            print(f'Missing appt: {fu_schedules[0].schedule_name}')
            break
        try:
            appt_2 = ChildAppointment.objects.get(
                subject_identifier=child, schedule_name=fu_schedules[1].schedule_name)
        except ChildAppointment.DoesNotExist:
            print(f'Missing appt: {fu_schedules[1].schedule_name}')
            break

        try:
            visit = ChildVisit.objects.get(appointment=appt_1)
        except ChildVisit.DoesNotExist:
            dt1 = appt_1.appt_datetime + relativedelta(months=12)

            if appt_2.appt_datetime <= dt1:
                print(child, appt_1.appt_datetime, appt_2.appt_datetime)
        else:
            dt1 = visit.report_datetime + relativedelta(months=12)
    
            if appt_2.appt_datetime <= dt1:
                print(child, visit.report_datetime, appt_2.appt_datetime)

100%|██████████| 470/470 [00:18<00:00, 25.39it/s]
